# k-nearest neighbour

An implementation of the k-nearest neighbour algorithm.

Sebastian Thomas

In [1]:
# python
import bisect
from collections import Counter

# data
import numpy as np

# machine learning
from sklearn.base import BaseEstimator, ClassifierMixin

In [2]:
class KNN(BaseEstimator, ClassifierMixin):
    
    def __init__(self, n_neighbours=5):
        self.n_neighbours = n_neighbours
        
    def _distance(self, x, y):
        return np.sqrt(np.sum((x - y)**2))
        
    def _neighbour_indices(self, x_pred):
        distances = []
        
        for (idx, x_train) in enumerate(self.X_train_):
            # insert dist in the sorted list distances
            bisect.insort(distances, (self._distance(x_pred, x_train), idx))
            # reduce length
            if len(distances) > self.n_neighbours:
                distances.pop()
            
        return [idx for (dist, idx) in distances]
    
    def _neighbour_targets(self, x_pred):
        return [self.y_train_[idx] for idx in self._neighbour_indices(x_pred)]

    def fit(self, X_train, y_train):
        self.X_train_ = X_train
        self.y_train_ = y_train
    
    def predict(self, X_pred):
        predictions = [Counter(self._neighbour_targets(x_pred)).most_common(1)[0][0] for x_pred in X_pred]        
        return np.array(predictions)

In [3]:
# example
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

(X, y) = load_iris(return_X_y=True)

(X_train, X_test, y_train, y_test) = train_test_split(X, y, random_state=0)

classifier = make_pipeline(StandardScaler(), KNN())
classifier.fit(X_train, y_train)

print('accuracy for k = 5:', accuracy_score(y_test, classifier.predict(X_test)))

accuracy for k = 5: 0.9736842105263158
